system mask 部分完整代码

In [9]:
import os

import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torchvision.transforms.functional import to_tensor, to_pil_image

In [4]:
from torch import unsqueeze


class Matting_forMask():
    def __init__(self, vision_num = 5):
        # self.num_vision = vision_num
        self.model = torch.jit.load('/home/wph/BackgroundMatting/TorchScript/torchscript_resnet50_fp32.pth').eval().cuda()
        

    def get_group_masks(self, src_tensor, bgr_tensor):
        print(src_tensor.dtype)
        pha = self.model(src_tensor, bgr_tensor)[0]
        
        # for i in range(5):
        #     src = src_tensor[i, :, :, :].unsqueeze(0)
        #     bgr = bgr_tensor[i, :, :, :].unsqueeze(0)
        #     print(src.dtype)
        #     pha = self.model(src, bgr)[0]
        #     print(pha.shape)
        #     phas.append(pha[0])
        
        # phas = torch.tensor(phas)
        return pha


In [6]:
imgs_list = [cv2.cvtColor(cv2.imread(f"../test_data/{i+1}.fg.jpg"), cv2.COLOR_BGR2RGB) for i in range(5)]
bgrs_list = [cv2.cvtColor(cv2.imread(f"../test_data/{i+1}.jpg"), cv2.COLOR_BGR2RGB) for i in range(5)]

# imgs_list = [cv2.imread(f"../test_data/{i+1}.fg.jpg") for i in range(5)]
# bgrs_list = [cv2.imread(f"../test_data/{i+1}.jpg") for i in range(5)]

imgs = np.stack(imgs_list, axis=0)
bgrs = np.stack(bgrs_list, axis=0)

imgs_tensor = torch.tensor(imgs).permute(0,3,1,2).float()
imgs_tensor = (imgs_tensor/255).cuda()
bgrs_tensor = torch.tensor(bgrs).permute(0,3,1,2)
bgrs_tensor = (bgrs_tensor/255).float().cuda()

In [7]:
# test get_fore_rect

def get_fore_rect(back, src):
    fgbg = cv2.createBackgroundSubtractorMOG2(history=1, varThreshold=100, detectShadows=False)
    
    # get the front mask
    mask = fgbg.apply(back)
    mask = fgbg.apply(src)

    # eliminate the noise
    # line = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 5), (-1, -1))
    # mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, line)

    max_area = -1
    max_rect = None
    # find the max area contours
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in range(len(contours)):
        area = cv2.contourArea(contours[c])
        if area < 150:
            continue
        rect = cv2.minAreaRect(contours[c])

        if area > max_area:
            max_area = area
            max_rect = rect

    box = cv2.boxPoints(max_rect)
    max_x, min_x = np.max(box[:,0]), np.min(box[:,0])
    max_y, min_y = np.max(box[:,1]), np.min(box[:,1])
    box = np.array([[min_x, max_y],[min_x, min_y],[max_x, min_y],[max_x, max_y]])
    box = np.int0(box)

    # min_row = int(box[1][1]/4) *4
    # max_row = int((box[0][1]+3)/4) *4
    # min_col = int(box[0][0]/4) *4
    # max_col = int((box[2][0]+3)/4) *4

    max_x, max_y = int((max_x+3)/4) *4, int((max_y+3)/4) *4
    min_x, min_y = int(min_x/4) *4, int(min_y/4) *4

    w = max_x - min_x
    h = max_y - min_y

    return (w, h, min_x, min_y)

crops = [get_fore_rect(imgs_list[i], bgrs_list[i]) for i in range(5)]
crops

[(1380, 1736, 1688, 384),
 (1168, 1564, 1764, 412),
 (1180, 1560, 1592, 592),
 (1496, 1836, 1224, 512),
 (1244, 1668, 1196, 420)]

In [16]:
matting = Matting_forMask()
pha = matting.get_group_masks(imgs_tensor, bgrs_tensor)

torch.float32


In [18]:
def get_masks(pha):
    masks = []
    # pha_np = []
    # for i in range(5):
    #     pha_np.append(pha[i].to(torch.device('cpu')).numpy())

    pha_np = pha.to(torch.device('cpu')).numpy()
    print("pha_np", pha_np.shape)

    for i in range(5):
        mask_np = np.transpose(pha_np[i], (1,2,0))
        mask = np.uint8(mask_np*255)
        masks.append(mask)
        # cv2.imwrite(f'./test_results/test_mask{i+1}.png', np.uint8(mask_np*255))
        # mask_output = cv2.cvtColor(np.uint8(mask_np*255), cv2.COLOR_BGR2RGB)
        # mask = np.uint8(mask_np*255)
        # masks.append(mask_output[:,:,0])
    
    return masks

masks = get_masks(pha)

pha_np (5, 1, 2160, 3840)


In [19]:
for i in range(5):
    cv2.imwrite(f'./test_results/test_float_mask{i+1}.png', masks[i])

In [20]:
float_mask = cv2.imread('./test_results/test_float_mask1.png')
mask = cv2.imread('./test_results/test_mask1.png')

for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        for k in range(3):
            if float_mask[i][j][k] != mask[i][j][k]: 
                print("different")

print("same")

same


In [11]:
imgs = [cv2.imread(f"/data/FastMVSNet/lab/Rectified/scan1/rect_00{i+1}_3_r5000.png") for i in range(5)]
for img in imgs:
    print(img.shape)

(672, 384, 3)
(672, 384, 3)
(672, 384, 3)
(672, 384, 3)
(672, 384, 3)
